In [ ]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
import psutil
import subprocess
import random
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
import re

order_IDs = []
ship_by_dates = []
deliver_by_dates = []
purchase_dates = []
fulfillment_methods = []
buyer_names = []
city_names = []
states = []
zip_codes = []
item_subtotals = []
tax_totals = []
refund_totals = []
grand_totals = []
payment_methods = []
# carriers = []
# shipping_services = []
# tax_collections = []
# tax_responsibles = []
statuses = []
product_names = []
ASINS = []
SKUS = []
conditionS = []
order_item_IDS = []
quantities = []
unit_prices = []
ship_dates = []
tracking_ids = []
package_types = []
dimensions = []
weights = []
shipping_labels = []
shipping_costs = []

def fetch_totals(driver):
    # Fetching Item Subtotal
    item_subtotal_xpath = '//*[@id="MYO-app"]/div/div[1]/div[2]/div[1]/table[1]/tbody/tr[1]/td[2]/span'
    item_subtotal_element = driver.find_element(By.XPATH, item_subtotal_xpath)
    item_subtotal = item_subtotal_element.text
    try:
        # Fetching Tax Total
        tax_total_row_xpath = '//*[@id="MYO-app"]/div/div[1]/div[2]/div[1]/table[1]/tbody//tr[contains(td/span, "Tax total")]'
        tax_total_element = driver.find_element(By.XPATH, tax_total_row_xpath)
        tax_total = tax_total_element.find_element(By.XPATH, './td[2]/span').text
    except NoSuchElementException:
        tax_total = 0

    try:
        # Fetching Refunds Total
        refunds_total_row_xpath = '//*[@id="MYO-app"]/div/div[1]/div[2]/div[1]/table[1]/tbody//tr[contains(td/span, "Refunds total")]'
        refunds_total_element = driver.find_element(By.XPATH, refunds_total_row_xpath)
        refunds_total = refunds_total_element.find_element(By.XPATH, './td[2]/span').text
    except NoSuchElementException:
        refunds_total = 0

    
    # If refunds total is present, set grand total to zero
    if refunds_total != 0:
        grand_total = 0
    else:
        # Fetching Grand Total
        grand_total_xpath = '//*[@id="MYO-app"]/div/div[1]/div[2]/div[1]/span/table/tbody/tr/td[2]/span'
        grand_total_element = driver.find_element(By.XPATH, grand_total_xpath)
        grand_total = grand_total_element.text
    
    return item_subtotal, tax_total, refunds_total, grand_total


driver = webdriver.Chrome()
driver.maximize_window() 
driver.get('https://sellercentral.amazon.com/orders-v3/mfn/shipped?mons_sel_dir_mcid=amzn1.merchant.d.ABAACLURLA6ESTROSUOBDTVKH4ZA&mons_sel_mkid=ATVPDKIKX0DER&mons_sel_dir_paid=amzn1.pa.d.ABUVOVY2TXHPYCUDJRIYRH6SOPNA&ignore_selection_changed=true&date-range=last-365&page=1')

# Wait for the email input field to load
wait = WebDriverWait(driver, 40)
email_input = wait.until(EC.presence_of_element_located((By.XPATH, "//*[@id='ap_email']")))

# Input email into the field
email_input.send_keys("enter_your_email_here")

# Wait for the email input field to load
wait = WebDriverWait(driver, 10)
pass_input = wait.until(EC.presence_of_element_located((By.XPATH, "//*[@id='ap_password']")))

# Input email into the field
pass_input.send_keys("enter_password")

time.sleep(5)
submit = wait.until(EC.presence_of_element_located((By.XPATH, "//*[@id='signInSubmit']")))
submit.click()

time.sleep(90)

# Get the text containing the number of orders
orders_text = wait.until(EC.presence_of_element_located((By.XPATH, "//*[@id='myo-sorting-bar']/div[1]/div/span[1]"))).text

# Extract only the numeric part using regular expressions
match = re.search(r'\d+', orders_text)
if match:
    num_products = int(match.group())
else:
    num_products = 0  # or any default value if no number is found
print("Number of Products are: ", num_products)

# Loop through each product row and fetch the text
for i in range(1, num_products + 1):

    driver.get('https://sellercentral.amazon.com/orders-v3/mfn/shipped?mons_sel_dir_mcid=amzn1.merchant.d.ABAACLURLA6ESTROSUOBDTVKH4ZA&mons_sel_mkid=ATVPDKIKX0DER&mons_sel_dir_paid=amzn1.pa.d.ABUVOVY2TXHPYCUDJRIYRH6SOPNA&ignore_selection_changed=true&date-range=last-365&page=1')
    options100 = wait.until(EC.presence_of_element_located((By.XPATH, "//*[@id='a-autoid-2']/span")))
    # time.sleep(2)
    options100.click()
    select100 = wait.until(EC.presence_of_element_located((By.XPATH, "//*[@id='a-popover-1']/div/div/ul/li[4]")))
    # time.sleep(2)
    select100.click()
    time.sleep(10)
    
    #Clicking Each ID of the Order
    id_xpath = f'//*[@id="orders-table"]/tbody/tr[{i}]/td[3]/div/div[1]/a'
    order_id = driver.find_element("xpath",id_xpath)
    order_id.click()

    time.sleep(5)
    
    print("Fetching Order # ", i, "...\n")

    try:
        # Fetching Order ID
        print("Fetching Order ID...")
        order_ID_xpath = f'//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[1]/div[1]/div/span[5]'
        order_ID_element = driver.find_element("xpath", order_ID_xpath)
        order_ID = order_ID_element.text
    except NoSuchElementException:
        print("Order ID not found.")
        order_ID = "N/A"
    finally:
        order_IDs.append(order_ID)

    try:
        # Fetching Ship By Date
        print("Fetching Ship By Date...")
        ship_by_xpath = f'//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[2]/div[1]/div/div/div/div/div/div[1]/table/tbody/tr[1]/td[2]/span'
        ship_by_date_element = driver.find_element("xpath", ship_by_xpath)
        ship_by_date = ship_by_date_element.text
    except NoSuchElementException:
        print("Ship By Date not found.")
        ship_by_date = "N/A"
    finally:
        ship_by_dates.append(ship_by_date)

    try:
        # Fetching Deliver By Date
        print("Fetching Deliver By Date...")
        deliver_by_xpath = f'//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[2]/div[1]/div/div/div/div/div/div[1]/table/tbody/tr[2]/td[2]/span'
        deliver_by_element = driver.find_element("xpath", deliver_by_xpath)
        deliver_by_date = deliver_by_element.text
    except NoSuchElementException:
        print("Deliver By Date not found.")
        deliver_by_date = "N/A"
    finally:
        deliver_by_dates.append(deliver_by_date)

    try:
        # Fetching Purchase Date
        print("Fetching Purchase Date...")
        purchase_xpath = f'//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[2]/div[1]/div/div/div/div/div/div[1]/table/tbody/tr[3]/td[2]/span'
        purchase_element = driver.find_element("xpath", purchase_xpath)
        purchase_date = purchase_element.text
    except NoSuchElementException:
        print("Purchase Date not found.")
        purchase_date = "N/A"
    finally:
        purchase_dates.append(purchase_date)

    try:
        # Fetching Fulfillment Methods
        print("Fetching Fulfillment Methods...")
        fulfillment_xpath = f'//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[2]/div[1]/div/div/div/div/div/div[2]/table/tbody/tr[2]/td[2]/span/span'
        fulfillment_element = driver.find_element("xpath", fulfillment_xpath)
        fulfillment_method = fulfillment_element.text
    except NoSuchElementException:
        print("Fulfillment Methods not found.")
        fulfillment_method = "N/A"
    finally:
        fulfillment_methods.append(fulfillment_method)
    
    try:
        # Fetching Buyer Name
        print("Fetching Buyer's Name...")
        buyer_xpath = f'//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[2]/div[2]/div/div/div/div/div[2]/div/table/tbody/tr/td[2]/span/b/a'
        buyer_element = driver.find_element("xpath", buyer_xpath)
        buyer_name = buyer_element.text
    except NoSuchElementException:
        print("Buyer's Name not found.")
        buyer_name = "N/A"
    finally:
        buyer_names.append(buyer_name)

    try:
        # Fetching City Name
        print("Fetching City Name...")
        city_xpath = f'//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[2]/div[2]/div/div/div/div/div[1]/table/tbody/tr[1]/td/span/span[3]/div/div/span[2]'
        city_element = driver.find_element("xpath", city_xpath)
        city_name = city_element.text
    except NoSuchElementException:
        print("City Name not found.")
        city_name = "N/A"
    finally:
        city_names.append(city_name)

    try:
        # Fetching State
        print("Fetching State...")
        state_xpath = f'//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[2]/div[2]/div/div/div/div/div[1]/table/tbody/tr[1]/td/span/span[3]/div/div/span[3]'
        state_element = driver.find_element("xpath", state_xpath)
        state = state_element.text
    except NoSuchElementException:
        print("State not found.")
        state = "N/A"
    finally:
        states.append(state)

    try:
        # Fetching Zip/Postal Code
        print("Fetching Zip Code...")
        zip_xpath = f'//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[2]/div[2]/div/div/div/div/div[1]/table/tbody/tr[1]/td/span/span[3]/div/div/span[4]'
        zip_element = driver.find_element("xpath", zip_xpath)
        zip_code = zip_element.text
    except NoSuchElementException:
        print("Zip Code not found.")
        zip_code = "N/A"
    finally:
        zip_codes.append(zip_code)
        time.sleep(2)
    
    #Fetching Totals
    print("Fetching Totals...")
    item_subtotal, tax_total, refunds_total, grand_total = fetch_totals(driver)
    item_subtotals.append(item_subtotal)
    tax_totals.append(tax_total)
    refund_totals.append(refunds_total)
    grand_totals.append(grand_total)
    print("Item Subtotal : ", item_subtotal)
    print("Tax Total : ", tax_total)
    print("Refund Total : ", refunds_total)
    print("Grand Total : ", grand_total)

    time.sleep(8)

    
    try:
        # Fetching Payment Methods
        print("Fetching Data...")
        payment_method_xpath = f'//*[@id="MYO-app"]/div/div[1]/div[2]/div[1]/div[2]/span[3]'
        payment_method_element = driver.find_element("xpath", payment_method_xpath)
        payment_method = payment_method_element.text
    except NoSuchElementException:
        print("Payment Methods not found.")
        payment_method = "N/A"
    finally:
        payment_methods.append(payment_method)

    # Fetching Shipping Carrier Name
    print("Fetching Data...")
    carrier_xpath = '//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[4]/div[1]/div/div/div/div[2]/div[1]/div/span[4]'
    try:
    	carrier_element = driver.find_element("xpath", carrier_xpath)
    	carrier = carrier_element.text
    except NoSuchElementException:
    	carrier = "N/A"
    carriers.append(carrier)

    # Fetching Shipping Service
    print("Fetching Data...")
    shipping_service_xpath = '//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[4]/div[1]/div/div/div/div[2]/div[2]/div/span[4]'
    try:
    	shipping_service_element = driver.find_element("xpath", shipping_service_xpath)
    	shipping_service = shipping_service_element.text
    except NoSuchElementException:
    	shipping_service = "N/A"
    shipping_services.append(shipping_service)
    
    # Find the element using XPath
    try:
        more_details_element = driver.find_element("xpath", '//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[5]/div/div/span/span')
        # Check if the text content is "More details"
        if more_details_element.text == "More details":
            more_details = 1
            # Fetching Tax Collection Model
            print("Fetching Data...")
            tax_collection_xpath = f'//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[5]/div/div/div/div[1]/div/span[4]'
            tax_collection_element = driver.find_element("xpath", tax_collection_xpath)
            tax_collection = tax_collection_element.text
            tax_collections.append(tax_collection)
            
            #Fetching Tax Collection Responsible Party
            print("Fetching Data...")
            tax_responsible_xpath = f'//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[5]/div/div/div/div[2]/div/span[4]'
            tax_responsible_element = driver.find_element("xpath", tax_responsible_xpath)
            tax_responsible = tax_responsible_element.text
            tax_responsibles.append(tax_responsible)
        else:
            more_details = 0
            tax_collection = "N/A"
            tax_collections.append(tax_collection)
            tax_responsible = "N/A"
            tax_responsibles.append(tax_responsible)
    except NoSuchElementException:
        # If the element is not found, set more_details to 0
        more_details = 0
        tax_collection = 'N/A'
        tax_collections.append(tax_collection)
        tax_responsible = 'N/A'
        tax_responsibles.append(tax_responsible)
        
    print("More details:", more_details)
        
    #Fetching Status
    print("Fetching Data...")
    if more_details == 1:
        status_xpath = '//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[7]/div/table/tbody/tr/td[1]/div/div[1]'
        status_element = driver.find_element("xpath", status_xpath)
        status = status_element.text
        statuses.append(status)
    else:
        status_xpath = '//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[6]/div/table/tbody/tr/td[1]/div/div[1]'
        status_element = driver.find_element("xpath", status_xpath)
        status = status_element.text
        statuses.append(status)
    

    #Fetching Product Name
    print("Fetching Data...")

    if more_details == 1:
        product_name_xpath = '//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[7]/div/table/tbody/tr/td[3]/div/div[1]/div/a/div'
        product_name_element = driver.find_element("xpath", product_name_xpath)
        product_name = product_name_element.text
        product_names.append(product_name)
    else: 
        product_name_xpath = '//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[6]/div/table/tbody/tr/td[3]/div/div[1]/div/a/div'
        product_name_element = driver.find_element("xpath", product_name_xpath)
        product_name = product_name_element.text
        product_names.append(product_name)

    # Similarly, handle other elements in a similar manner...

    # Fetching ASIN
    print("Fetching Data...")
    if more_details == 1:
        ASIN_xpath = '//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[7]/div/table/tbody/tr/td[3]/div/div[2]/div/b'
        ASIN_element = driver.find_element("xpath", ASIN_xpath)
        ASIN = ASIN_element.text
        ASINS.append(ASIN)
    else:
        ASIN_xpath = '//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[6]/div/table/tbody/tr/td[3]/div/div[2]/div/b'
        ASIN_element = driver.find_element("xpath", ASIN_xpath)
        ASIN = ASIN_element.text
        ASINS.append(ASIN)

    # Fetching SKU
    print("Fetching Data...")
    if more_details == 1:
        SKU_xpath = '//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[7]/div/table/tbody/tr/td[3]/div/div[3]/div'
        SKU_element = driver.find_element("xpath", SKU_xpath)
        SKU = SKU_element.text
        SKUS.append(SKU)
    else:
        SKU_xpath = '//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[6]/div/table/tbody/tr/td[3]/div/div[3]/div'
        SKU_element = driver.find_element("xpath", SKU_xpath)
        SKU = SKU_element.text
        SKUS.append(SKU)

    # Fetching Condition
    print("Fetching Data...")
    if more_details == 1:
        condition_xpath = '//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[7]/div/table/tbody/tr/td[4]/div[1]/div/span[2]'
        condition_element = driver.find_element("xpath", condition_xpath)
        condition = condition_element.text
        conditionS.append(condition)
    else:
        condition_xpath = '//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[6]/div/table/tbody/tr/td[4]/div[1]/div/span[2]'
        condition_element = driver.find_element("xpath", condition_xpath)
        condition = condition_element.text
        conditionS.append(condition)

    # Fetching Order Item ID
    print("Fetching Data...")
    if more_details == 1:
        order_item_ID_xpath = '//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[7]/div/table/tbody/tr/td[4]/div[2]/div'
        order_item_ID_element = driver.find_element("xpath", order_item_ID_xpath)
        order_item_ID = order_item_ID_element.text
        order_item_IDS.append(order_item_ID)
    else:
        order_item_ID_xpath = '//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[6]/div/table/tbody/tr/td[4]/div[2]/div'
        order_item_ID_element = driver.find_element("xpath", order_item_ID_xpath)
        order_item_ID = order_item_ID_element.text
        order_item_IDS.append(order_item_ID)

    # Fetching Quantity
    print("Fetching Data...")
    if more_details == 1:
        quantity_xpath = '//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[7]/div/table/tbody/tr/td[5]'
        quantity_element = driver.find_element("xpath", quantity_xpath)
        quantity = quantity_element.text
        quantities.append(quantity)
    else:
        quantity_xpath = '//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[6]/div/table/tbody/tr/td[5]'
        quantity_element = driver.find_element("xpath", quantity_xpath)
        quantity = quantity_element.text
        quantities.append(quantity)
    
    # Fetching Unit Price
    print("Fetching Data...")
    unit_price_xpaths = [
        '//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[7]/div/table/tbody/tr/td[6]/span',
        '//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[6]/div/table/tbody/tr/td[6]/span'
    ]
    unit_price = None

    for xpath in unit_price_xpaths:
        try:
            unit_price_element = driver.find_element("xpath", xpath)
            unit_price = unit_price_element.text
            break  
        except NoSuchElementException:
            continue 
            
	# If unit price is still None, assign item_subtotal to unit_price
    if unit_price is None:
        unit_price = item_subtotal
        
    unit_prices.append(unit_price)
    
    # Fetching Ship Date
    print("Fetching Data...")
    if more_details == 1:
        ship_date_xpath = '//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[8]/div/div/div[1]/div[2]/div/div[1]/div/div[2]'
        ship_date_element = driver.find_element("xpath", ship_date_xpath)
        ship_date = ship_date_element.text
        ship_dates.append(ship_date)
    else:
        ship_date_xpath = '//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[7]/div/div/div[1]/div[2]/div/div[1]/div/div[2]'
        ship_date_element = driver.find_element("xpath", ship_date_xpath)
        ship_date = ship_date_element.text
        ship_dates.append(ship_date)

    # Fetching Tracking ID
    print("Fetching Data...")
    if more_details == 1:
        tracking_id_xpath = '//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[8]/div/div/div[1]/div[2]/div/div[2]/div[2]/div[2]'
        tracking_id_element = driver.find_element("xpath", tracking_id_xpath)
        tracking_id = str(tracking_id_element.text)
        tracking_ids.append(tracking_id)
    else:
        tracking_id_xpath = '//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[7]/div/div/div[1]/div[2]/div/div[2]/div[2]/div[2]'
        tracking_id_element = driver.find_element("xpath", tracking_id_xpath)
        tracking_id = str(tracking_id_element.text)
        tracking_ids.append(tracking_id)

        # Fetching Package Type
        print("Fetching Data...")
        try:
            if more_details == 1:
                package_type_xpath = '//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[8]/div/div/div[1]/div[3]/div/div/div[2]/div[2]/div[1]/div/div[2]/span'
            else:
                package_type_xpath = '//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[7]/div/div/div[1]/div[3]/div/div/div[2]/div[2]/div[1]/div/div[2]/span'
            package_type_element = driver.find_element("xpath", package_type_xpath)
            package_type = package_type_element.text
            package_types.append(package_type)
        except NoSuchElementException:
            package_type = "N/A"
            print("Package type element not found.")

    # Fetching Package Dimension (LWH)
        print("Fetching Data...")
        try:
            if more_details == 1:
                dimension_xpath = '//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[8]/div/div/div[1]/div[3]/div/div/div[2]/div[2]/div[2]/div/div[2]'
            else:
                dimension_xpath = '//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[7]/div/div/div[1]/div[3]/div/div/div[2]/div[2]/div[2]/div/div[2]'
            dimension_element = driver.find_element("xpath", dimension_xpath)
            dimension = dimension_element.text
            dimensions.append(dimension)
        except NoSuchElementException:
            dimension = "N/A"
            print("Package dimension element not found.")

    # Fetching Package Weight
        print("Fetching Data...")
        try:
            if more_details == 1:
                weight_xpath = '//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[8]/div/div/div[1]/div[3]/div/div/div[2]/div[2]/div[3]/div/div[2]'
            else:
                weight_xpath = '//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[7]/div/div/div[1]/div[3]/div/div/div[2]/div[2]/div[3]/div/div[2]'
            weight_element = driver.find_element("xpath", weight_xpath)
            weight = weight_element.text
            weights.append(weight)
        except NoSuchElementException:
            weight = 0
            print("Package weight element not found.")

    # Fetching Shipping Label
        print("Fetching Data...")
        try:
            if more_details == 1:
                shipping_label_xpath = '//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[8]/div/div/div[1]/div[3]/div/div/div[2]/div[3]/div/div[1]/div[2]/span'
            else:
                shipping_label_xpath = '//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[7]/div/div/div[1]/div[3]/div/div/div[2]/div[3]/div/div[1]/div[2]/span'
            shipping_label_element = driver.find_element("xpath", shipping_label_xpath)
            shipping_label = shipping_label_element.text
            shipping_labels.append(shipping_label)
        except NoSuchElementException:
            shipping_label = 0
            print("Shipping label element not found.")

    # Fetching Total Shipping Cost
        try:
            if more_details == 1:
                shipping_cost_xpath = '//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[8]/div/div/div[1]/div[3]/div/div/div[2]/div[3]/div/div[2]/div[2]/span'
            else:
                shipping_cost_xpath = '//*[@id="MYO-app"]/div/div[1]/div[1]/div/div[7]/div/div/div[1]/div[3]/div/div/div[2]/div[3]/div/div[2]/div[2]/span'
            shipping_cost_element = driver.find_element("xpath", shipping_cost_xpath)
            shipping_cost = shipping_cost_element.text
            shipping_costs.append(shipping_cost)
        except NoSuchElementException:
            shipping_cost = 0
            print("Shipping cost element not found.")

    time.sleep(2)
    # Writing to CSV
    with open('Orderwise_Shipped_Orders.csv', 'a', newline='') as csvfile:  # Open in append mode
        fieldnames = ['Order ID', 'Tracking ID', 'Product Name',
                      'Unit Price', 'Ship By Date',
                      'Deliver By Date', 'Purchase Date',
                      'Ship Date', 'Payment Method',
                      'Fulfillment Methods', 'ASIN', 'Quantities',
                      'Item Subtotals', 'Tax Total',
                      'Refund Total', 'Shipping Label',
                      'Shipping Cost', 'Grand Total',
                      'Status',
                      'Buyer Name', 'City Name', 'State',  'Zip Code',
                      'Carrier', 'Shipping Service',
                      'Tax Collection',  'Tax Responsible', 
                      'SKU',  'Condition',   'Order Item ID', 'Package Type','Dimensions','Weight']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        time.sleep(2)
        
        writer.writerow({
                'Order ID': order_ID,
                'Tracking ID': tracking_id,
                'Product Name': product_name, 
                'Unit Price': unit_price,
                'Ship By Date': ship_by_date, 
                'Deliver By Date': deliver_by_date,
                'Purchase Date': purchase_date,
                'Ship Date': ship_date,
                'Payment Method': payment_method,
                'Fulfillment Methods': fulfillment_method,
                'ASIN': ASIN,
                'Quantities': quantity,
                'Item Subtotals': item_subtotal, 
                'Tax Total': tax_total,
                'Refund Total': refunds_total,
                'Shipping Label': shipping_label,
                'Shipping Cost': shipping_cost,
                'Grand Total': grand_total,
                'Status': status, 
                'Buyer Name': buyer_name,
                'City Name': city_name,
                'State': state,
                'Zip Code': zip_code,
                'Carrier': carrier,
                'Shipping Service': shipping_service,
                'Tax Collection': tax_collection,
                'Tax Responsible': tax_responsible,
                'SKU': SKU,
                'Condition': condition,
                'Order Item ID': order_item_ID,
                'Package Type': package_type,
                'Dimensions': dimension,
                'Weight': weight
            })
        print("Data appended to CSV file after iteration ", i+1)
        # time.sleep(5)
    
    print("CSV file 'Orderwise_Shipped_Orders.csv' has been updated successfully.")

    time.sleep(2)

    print("Order # ", i+1, " fetched successfully!")

    if i == 100:
        i = 0
        num_products = num_products - 100
        #Page 2 when 100 items are done
        next_page = wait.until(EC.presence_of_element_located((By.XPATH, "//*[@id='myo-layout']/div[2]/div[4]/div/div/div[1]/div[2]/div/ul/li[2]/a")))
        next_page.click()
        time.sleep(10)
        

# Writing to CSV
with open('Shipped_Orders.csv', 'w', newline='') as csvfile:
    fieldnames = ['Order ID', 'Tracking ID', 'Product Name',
                  'Unit Price', 'Ship By Date',
                  'Deliver By Date', 'Purchase Date',
                  'Ship Date', 'Payment Method',
                  'Fulfillment Methods', 'ASIN', 'Quantities',
                  'Item Subtotals', 'Tax Total',
                  'Refund Total', 'Shipping Label',
                  'Shipping Cost', 'Grand Total',
                  'Status',
                  'Buyer Name', 'City Name', 'State',  'Zip Code',
                  'Carrier', 'Shipping Service',
                  'Tax Collection',  'Tax Responsible', 
                  'SKU',  'Condition',   'Order Item ID', 'Package Type','Dimensions','Weight']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for i in range(len(order_item_IDS)):
        writer.writerow({
            'Order ID': order_IDs[i],
            'Tracking ID': tracking_ids[i],
            'Product Name': product_names[i], 
            'Unit Price': unit_prices[i],
            'Ship By Date': ship_by_dates[i], 
            'Deliver By Date': deliver_by_dates[i],
            'Purchase Date': purchase_dates[i],
            'Ship Date': ship_dates[i],
            'Payment Method': payment_methods[i],
            'Fulfillment Methods': fulfillment_methods[i],
            'ASIN': ASINS[i],
            'Quantities': quantities[i],
            'Item Subtotals': item_subtotals[i], 
            'Tax Total': tax_totals[i],
            'Refund Total': refund_totals[i],
            'Shipping Label': shipping_labels[i],
            'Shipping Cost': shipping_costs[i],
            'Grand Total': grand_totals[i],
            'Status': statuses[i], 
            'Buyer Name': buyer_names[i],
            'City Name': city_names[i],
            'State': states[i],
            'Zip Code': zip_codes[i],
            'Carrier': carriers[i],
            'Shipping Service': shipping_services[i],
            'Tax Collection': tax_collections[i],
            'Tax Responsible': tax_responsibles[i],
            'SKU': SKUS[i],
            'Condition': conditionS[i],
            'Order Item ID': order_item_IDS[i],
            'Package Type': package_types[i],
            'Dimensions': dimensions[i],
            'Weight': weights[i]
        })
        
print("CSV file 'Shipped_Orders.csv' has been created successfully.")
